<a href="https://colab.research.google.com/github/GiovaniMicheloni/mlearning-w-python/blob/main/cross_val_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Validação cruzada dos modelos para a base de crédito

In [1]:
!pip -q install plotly

In [2]:
!pip -q  install yellowbrick

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
basecredito = pd.read_csv('/content/credit_data.csv')

In [6]:
basecredito.describe()

,clientid,income,age,loan,default
count,2000.000000,2000.000000,1997.000000,2000.000000,2000.000000
mean,1000.500000,45331.600018,40.807559,4444.369695,0.141500
std,577.494589,14326.327119,13.624469,3045.410024,0.348624
min,1.000000,20014.489470,-52.423280,1.377630,0.000000
25%,500.750000,32796.459717,28.990415,1939.708847,0.000000
50%,1000.500000,45789.117313,41.317159,3974.719419,0.000000
75%,1500.250000,57791.281668,52.587040,6432.410625,0.000000
max,2000.000000,69995.685578,63.971796,13766.051239,1.000000


In [7]:
basecredito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   clientid  2000 non-null   int64  
 1   income    2000 non-null   float64
 2   age       1997 non-null   float64
 3   loan      2000 non-null   float64
 4   default   2000 non-null   int64  
dtypes: float64(3), int64(2)
memory usage: 78.3 KB


In [8]:
basecredito[basecredito['age']<0]

,clientid,income,age,loan,default
15,16,50501.726689,-28.218361,3977.287432,0
21,22,32197.620701,-52.423280,4244.057136,0
26,27,63287.038908,-36.496976,9595.286289,0


In [9]:
basecredito[basecredito['age']>0].mean()

,0
clientid,1003.431795
income,45328.856915
age,40.927700
loan,4443.240892
default,0.141926


In [10]:
basecredito.loc[basecredito['age']<0, 'age'] = 40.92

In [11]:
basecredito[basecredito['age']<0]

,clientid,income,age,loan,default


In [12]:
basecredito.isnull().sum()

,0
clientid,0
income,0
age,3
loan,0
default,0


In [13]:
basecredito.loc[pd.isnull(basecredito['age'])]

,clientid,income,age,loan,default
28,29,59417.805406,NaN,2082.625938,0
30,31,48528.852796,NaN,6155.784670,0
31,32,23526.302555,NaN,2862.010139,0


In [14]:
basecredito['age']= basecredito['age'].fillna(basecredito['age'].mean())

In [15]:
xcredito = basecredito.iloc[:,1:4].values
xcredito

array([[6.61559251e+04, 5.90170151e+01, 8.10653213e+03],
       [3.44151540e+04, 4.81171531e+01, 6.56474502e+03],
       [5.73171701e+04, 6.31080495e+01, 8.02095330e+03],
       ...,
       [4.43114493e+04, 2.80171669e+01, 5.52278669e+03],
       [4.37560566e+04, 6.39717958e+01, 1.62272260e+03],
       [6.94365796e+04, 5.61526170e+01, 7.37883360e+03]])

In [16]:
ycredito = basecredito.iloc[:,4].values
ycredito

array([0, 0, 0, ..., 1, 0, 0])

In [17]:
from sklearn.preprocessing import StandardScaler
standardscaler = StandardScaler()
xcredito = standardscaler.fit_transform(xcredito)
xcredito

array([[ 1.45393393,  1.36538093,  1.20281942],
       [-0.76217555,  0.5426602 ,  0.69642695],
       [ 0.83682073,  1.67417189,  1.17471147],
       ...,
       [-0.07122592, -0.97448519,  0.35420081],
       [-0.11000289,  1.73936739, -0.92675625],
       [ 1.682986  ,  1.14917639,  0.96381038]])

In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

In [19]:
from sklearn.model_selection import cross_val_score, KFold

In [22]:
resultados_arvore = []
resultados_random_forest = []
resultados_knn = []
resultados_logistica = []
resultados_svm = []
resultados_rede_neural = []

for i in range(30):
  print(i)
  kfold = KFold(n_splits=10, shuffle=True, random_state=i)

  arvore = DecisionTreeClassifier(criterion='entropy', min_samples_leaf=1, min_samples_split=5, splitter='best')
  scores = cross_val_score(arvore, xcredito, ycredito, cv = kfold)
  #print(scores)
  #print(scores.mean())
  resultados_arvore.append(scores.mean())

  random_forest = RandomForestClassifier(criterion = 'entropy', min_samples_leaf = 1, min_samples_split=5, n_estimators = 10)
  scores = cross_val_score(random_forest, xcredito, ycredito, cv = kfold)
  resultados_random_forest.append(scores.mean())

  knn = KNeighborsClassifier()
  scores = cross_val_score(knn, xcredito, ycredito, cv = kfold)
  resultados_knn.append(scores.mean())

  svm = SVC(kernel = 'rbf', C = 2.0)
  scores = cross_val_score(svm, xcredito, ycredito, cv = kfold)
  resultados_svm.append(scores.mean())

  rede_neural = MLPClassifier(activation = 'relu', batch_size = 56, solver = 'adam')
  scores = cross_val_score(rede_neural, xcredito, ycredito, cv = kfold)
  resultados_rede_neural.append(scores.mean())

0


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptro

1


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptro

2


...

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptro

28


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptro

29


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptro

In [24]:
resultados_arvore

[np.float64(0.984),
 np.float64(0.9894999999999999),
 np.float64(0.9875),
 np.float64(0.9855),
 np.float64(0.991),
 np.float64(0.9870000000000001),
 np.float64(0.9890000000000001),
 np.float64(0.9889999999999999),
 np.float64(0.9879999999999999),
 np.float64(0.986),
 np.float64(0.9889999999999999),
 np.float64(0.9865),
 np.float64(0.9879999999999999),
 np.float64(0.9879999999999999),
 np.float64(0.9879999999999999),
 np.float64(0.9899999999999999),
 np.float64(0.9894999999999999),
 np.float64(0.9855),
 np.float64(0.986),
 np.float64(0.9889999999999999),
 np.float64(0.9854999999999998),
 np.float64(0.9884999999999999),
 np.float64(0.9869999999999999),
 np.float64(0.9884999999999999),
 np.float64(0.9869999999999999),
 np.float64(0.9884999999999998),
 np.float64(0.9880000000000001),
 np.float64(0.99),
 np.float64(0.9884999999999999),
 np.float64(0.9890000000000001)]

In [25]:
resultados_random_forest

[np.float64(0.984),
 np.float64(0.986),
 np.float64(0.9854999999999998),
 np.float64(0.9839999999999998),
 np.float64(0.9835),
 np.float64(0.9884999999999999),
 np.float64(0.986),
 np.float64(0.9844999999999999),
 np.float64(0.9875),
 np.float64(0.985),
 np.float64(0.982),
 np.float64(0.9879999999999999),
 np.float64(0.985),
 np.float64(0.985),
 np.float64(0.978),
 np.float64(0.986),
 np.float64(0.9845),
 np.float64(0.9834999999999999),
 np.float64(0.9800000000000001),
 np.float64(0.9795),
 np.float64(0.984),
 np.float64(0.9814999999999999),
 np.float64(0.9834999999999999),
 np.float64(0.9865),
 np.float64(0.9815000000000002),
 np.float64(0.9834999999999999),
 np.float64(0.982),
 np.float64(0.982),
 np.float64(0.9804999999999999),
 np.float64(0.9814999999999999)]

In [26]:
resultados_knn

[np.float64(0.977),
 np.float64(0.983),
 np.float64(0.9814999999999999),
 np.float64(0.9784999999999998),
 np.float64(0.9819999999999999),
 np.float64(0.9809999999999999),
 np.float64(0.978),
 np.float64(0.9804999999999998),
 np.float64(0.9795),
 np.float64(0.9799999999999999),
 np.float64(0.9824999999999999),
 np.float64(0.9810000000000001),
 np.float64(0.9839999999999998),
 np.float64(0.9809999999999999),
 np.float64(0.984),
 np.float64(0.9819999999999999),
 np.float64(0.9839999999999998),
 np.float64(0.982),
 np.float64(0.9804999999999999),
 np.float64(0.9774999999999998),
 np.float64(0.982),
 np.float64(0.9814999999999999),
 np.float64(0.9789999999999999),
 np.float64(0.9804999999999999),
 np.float64(0.9795),
 np.float64(0.9819999999999999),
 np.float64(0.9810000000000001),
 np.float64(0.9804999999999999),
 np.float64(0.9810000000000001),
 np.float64(0.9800000000000001)]

In [27]:
resultados_svm

[np.float64(0.9849999999999998),
 np.float64(0.9860000000000001),
 np.float64(0.9854999999999998),
 np.float64(0.9855),
 np.float64(0.9844999999999999),
 np.float64(0.9834999999999999),
 np.float64(0.984),
 np.float64(0.9865),
 np.float64(0.9869999999999999),
 np.float64(0.9844999999999999),
 np.float64(0.985),
 np.float64(0.985),
 np.float64(0.9865),
 np.float64(0.9884999999999999),
 np.float64(0.9869999999999999),
 np.float64(0.9854999999999998),
 np.float64(0.9849999999999998),
 np.float64(0.9869999999999999),
 np.float64(0.985),
 np.float64(0.982),
 np.float64(0.9835),
 np.float64(0.985),
 np.float64(0.9860000000000001),
 np.float64(0.985),
 np.float64(0.986),
 np.float64(0.9844999999999999),
 np.float64(0.984),
 np.float64(0.9864999999999998),
 np.float64(0.985),
 np.float64(0.984)]

In [28]:
resultados_rede_neural

[np.float64(0.9964999999999999),
 np.float64(0.9974999999999999),
 np.float64(0.9970000000000001),
 np.float64(0.9970000000000001),
 np.float64(0.9964999999999999),
 np.float64(0.9964999999999999),
 np.float64(0.9970000000000001),
 np.float64(0.9969999999999999),
 np.float64(0.9969999999999999),
 np.float64(0.9964999999999998),
 np.float64(0.9969999999999999),
 np.float64(0.9970000000000001),
 np.float64(0.9964999999999999),
 np.float64(0.9964999999999999),
 np.float64(0.9974999999999999),
 np.float64(0.9974999999999999),
 np.float64(0.9964999999999999),
 np.float64(0.9974999999999999),
 np.float64(0.9969999999999999),
 np.float64(0.9970000000000001),
 np.float64(0.9969999999999999),
 np.float64(0.9959999999999999),
 np.float64(0.9969999999999999),
 np.float64(0.9959999999999999),
 np.float64(0.9964999999999999),
 np.float64(0.9969999999999999),
 np.float64(0.9970000000000001),
 np.float64(0.9964999999999999),
 np.float64(0.9964999999999999),
 np.float64(0.9969999999999999)]